**Задание 1: Векторизация текста с использованием Мешка слов (BoW) и TF-IDF**

1. Создайте корпус из минимум 10 текстовых документов (можно взять новостные статьи, отзывы на товары, статьи по одной тематике).
2. Реализуйте предобработку текста, включающую:
* Приведение к нижнему регистру
* Токенизацию
* Удаление стоп-слов и пунктуации
* Лемматизацию/стемминг
3. Реализуйте модель Мешка слов (BoW) с использованием CountVectorizer из scikit-learn.
4. Реализуйте модель TF-IDF с использованием TfidfVectorizer из scikit-learn.
5. Найдите 10 самых значимых терминов для каждого документа по обоим подходам и сравните результаты.
6. Визуализируйте сходство документов с помощью метрики косинусного расстояния для обоих подходов.
7. Прокомментируйте разницу в результатах между BoW и TF-IDF

In [5]:
import re
import string
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

texts = [
        "Реальная жизнь редко делится на «чёрное» и «белое», так же, как и люди — на исключительно «плохих» или «хороших» ... Дорама поведает историю двух совершенно разных людей. Су Ëль — «плохой парень», потерявший веру в человечество, но при этом считающийся отличным полицейским, успешно раскрывающим одно дело за другим. Однако встреча с его полной противоположностью — «праведным» человеком, скрывающимся за псевдонимом «Кей», но творящим ужасные вещи, — полностью меняет ход событий... Су Ëль получил лишь аттестат о среднем образовании, что, однако, не помешало харизматичному и обаятельному молодому человеку дослужиться в полиции до ведущего сотрудника отдела по борьбе с распространением запрещённых веществ. Однако движет героем совсем не стремление к справедливости: мужчина амбициозен и предпочитает закрывать глаза на те преступления, раскрытие которых не способно принести ему продвижения по карьерной лестнице и одобрения от начальства. Но в тот самый момент, когда на горизонте замаячило долгожданное повышение на перспективное место, в городе объявился сумасшедший преступник, светлое будущее стало ускользать от него... Теперь же Су Ëль готов выйти на тропу войны и во что бы то ни стало разобраться с врагом, который помешал его планам! Но что случится, если интересы обоих пересекутся в стремлении к одной и той же цели?",
        "Сюжет дорамы Мышь раскрывает захватывающую историю, действие которой происходит в мире, в котором люди могут идентифицировать психопатов с помощью ДНК-теста плода в утробе матери. Чон Ба Рым - честный и добрый новичок полицейский, верящий в справедливость. После столкновения с серийным убийцей-психопатом, чьи безжалостные убийства навели ужас на всю страну, его жизнь полностью меняется. Го Му Чи - опытный детектив, родители которого были убиты, когда он был юн. С тех пор цель его жизни – найти убийцу и отомстить за смерть родителей. Для поимки преступников Го Му Чи готов прибегнуть к любым средствам, не заботясь ни о законе, ни о правилах. О Бон И - проблемная старшеклассница, живущая с бабушкой и хорошо разбирающаяся в боевых искусствах. Каждый раз, когда она сталкивается с соседом-полицейским Чон Ба Рымом, они начинают забавно препираться друг с другом. Чхве Хон Чжу - талантливый режиссёр-продюсер, известная как «Шерлок Хон Чжу». Она также хранит секрет, который в детстве связал её с убийцей.",
        "Молодой парень Юн Чон Ву родом из небольшой деревни получает предложение о работе от своего друга из Сеула. Упускать такую возможность герой не собирается и пакует чемоданы, тем более что его девушка давно живет и работает здесь. Суета большого города и цены намного выше привычных не оставляют Юн Чон Ву другого выбора, как заселиться в самое дешевое общежитие, которое находится у черта на рогах. Дорама «Незнакомцы из ада» поведает вам, как не сойти с ума, живя со странными соседями, которые похожи на извращенцев, маньяков и умалишённых. Юн Чон Ву они настораживают, но владелица кажется ему приятной женщиной, и он решает остаться на полгода, пока не накопит достаточно средств для съемной квартиры. Еще одной неприятной личностью является бандит Ан Хи Джуном, который кажется герою самым адекватным человеком. Однажды на кухне он пытается предупредить Юн Чон Ву о том, что нужно бежать из этого места, но закончить разговор им не удается. Близнецы с отклонениями устраивают переполох, и тихой жизни здесь явно не будет. Между тем главный герой пишет книгу, а атмосфера, царящая вокруг него, мало способствует для ее написания. На утро отправившись на работу в небольшую компанию, где директором является его друг, Юн Чон Ву видит город в других красках. Желание возвращаться домой после работы у парня отсутствует, и он идет выпить с коллегами, но замечает драку на улице и решает вмешаться. Оказывается, герой, будучи солдатом в армии сталкивался с похожей ситуацией и теперь у него психологическая травма. Добравшись до своей комнаты, Юн Чон Ву находит странный ежедневник, который предположительно принадлежит бывшему жильцу, который совершил самоубийство. В нем он находит странные записи…",
        "Главная героиня Ли На Ми работает маркетологом в молодой компании. В качестве члена команды амбициозного стартапа девушке приходится много работать на благо дела. Целыми днями она трудится в офисе, а по вечерам подрабатывает в кафе «Мици», принадлежащем её отцу. Однажды после тяжёлого дня героиня ослабляет бдительность и случайно теряет смартфон в автобусе. Когда девушка осознаёт случившееся и пускается на поиски пропажи, оказывается уже слишком поздно: некто неизвестный успевает подобрать телефон девушки. Но Ли На Ми не сдаётся и решает позвонить на свой телефон с номера подруги для того, чтобы попросить нашедшего вернуть ей собственность. Неожиданно для На Ми, вор соглашается отдать ей телефон и предлагает забрать телефон из мастерской, куда он отдал его. Девушка, не подозревая подвоха, отправляется на встречу. Однако парень по имени О Джун Ëн, нашедший телефон, уже замыслил недоброе. Он получает доступ к паролю девушки и под видом починки устанавливает в телефон шпионское программное обеспечение. И как только телефон возвращается к законной владелице, то тут же начинает следить за каждым действием На Ми, включая звонки, сообщения и передвижения девушки. Медленно, но верно Джун Ëн проникает в жизнь На Ми. А со временем и становится причиной ссоры между девушкой и её отцом. Однако это лишь начало истории. Не догадываясь о происходящем, На Ми обращается к новому знакомому за помощью в исправлении ошибки телефона. И Джун Ëн вновь становится причиной её проблем, в которые оказывается вовлечены и самые близкие люди девушки. Тем временем полиция расследует серию леденящих душу убийств. Следствие связывает их с действиями одного и того же маньяка, отличительной чертой который коллекционирует телефоны своих жертв. А подозреваемым по делу проходит некий О Джун Ëн...",
        "Эта история о мести, которую совершает человек под натиском обстоятельств. О людях, не боящихся пойти против системы. В первую очередь это история Чжи Ёна, курсанта, обучающегося в полицейской академии. В юности гангстеры напали и убили его маму. Он жестоко разочаровался в справедливости правосудия своей страны. Ведь преступники, убившие его родителей, ушли от наказания. Они так и не были пойманы сотрудниками правоохранительной системы. Тогда Чжи Ён решает взять правосудие в свои руки. Поступает в полицейскую академию, чтобы найти преступников, лишивших его семьи. Днём он обычный студент, который прилежно учится. В свободное время он превращается в человека, у которого нет власти, но есть цель. Он занимается выслеживанием преступников, безнаказанно живущих в обществе. Мститель найдёт тех, кому удалось уйти от правосудия, либо наказание для них было слишком снисходительным. Но его деятельность начинает привлекать очень много внимания посторонних людей. О нём начинают говорить высокопоставленные личности. Про тайную войну с преступным миром пишут статьи в разных газетах. Борьба с несправедливостью становится довольно обсуждаемой в обществе. Однажды репортёру Чхвэ Ми Рё удаётся поймать мстителя с поличным во время его охоты на преступников. Однако вместо того, чтобы предать личность охотника на бандитов огласке. Журналист решает помочь ему в поиске правосудия для злодеев. Но так же личностью «Линчевателя» начинает интересоваться Сеульский следователь — Чон Хон. Он является главой столичного следственного отдела. Отслеживает существование тёмного героя и намеревается его найти всеми доступными способами.",
        "Апокалиптический триллер, действие которого происходит во времена, когда инфекционные заболевания стали нормой. В дораме показана классовая дискриминация, тонкие психологические войны и ожесточённая борьба за выживание, происходящие в многоэтажном жилом комплексе, изолированном от внешнего мира из-за нового вида инфекционного заболевания. Юн Сэ Бом – агент специального отряда полиции. Она решительна, поэтому быстро принимает решения и без колебаний претворяет их в жизнь. Но как только она осуществит свою мечту о переезде в новую квартиру, она столкнётся с беспрецедентным кризисом. Чон И Хён – умный и честный детектив из отдела по насильственным преступлениям, работающий на благо общества, а не для личной выгоды. Он всегда был влюблён в свою одноклассницу-друга Юн Сэ Бом. Чон И Хён попадает в водоворот событий после того, как принимает неожиданное предложение от Юн Сэ Бом. Хан Тэ Сок – руководитель медицинского подразделения вооружённых сил. Он тот, кто владеет «ключом» к новому инфекционному заболеванию. Ему было поручено определить причину нового инфекционного заболевания и предотвратить его распространение. Хан Тэ Сок - персонаж, который мгновенно трансформируется, поэтому его трудно «прочитать».",
        "Кан Бит На – проблематичная судья, которая скептически относится к неискреннему раскаянию преступников и не способна сопереживать боли и страданиям жертв. Судья Кан красива и имеет элитное происхождение, но её истинная сущность – демон, пришедший из ада, миссия которого состоит в том, чтобы наказать тех, кто не испытывает угрызений совести, но несёт ответственность за смерть других людей. Её долг – отправить их в ад, убив. Судья Кан знакомится с детективом Ха Да Оном, человеком с теплым характером, который ставит благополучие жертвы на первое место и помогает Кан Бит На осознать важность сочувствия и сострадания.",
        "Порой, все мы упускаем самое важное в жизни. Ведь каждый человек хочет не просто жить, но и ощущать ценность своей жизни, чувствовать, что его существование, его деятельность, жизненные позиции, преодоление препятствий, устремлённость в будущее несут какой-то смысл. В обыденных ситуациях мы редко осознаём, что нам нужна не только жизнь сама по себе, но и её осмысленность. Естественно, что у всех своя проблема, которая является предпосылкой, причиной обесценивания самого важного — жизни. Именно когда человек перестаёт получать удовольствие от жизни, у него появляются психологические расстройства. Чхве И Джэ — несчастный человек, ибо вся его жизнь полка неудач и горечи. Долгие годы поисков работы не приносят никаких плодов. Столкнувшись с мошенниками в сфере биткойнов, ему приходится распрощаться со всеми сбережениями. К тому же и любящая девушка внезапно оставляет его одного в трудную минуту. Обременённый тяжёлыми мыслями, не справляющийся с давлением требовательного общества неудачник прибегает к самоубийству. Но в его случае умереть так просто не получается. Оскорблённая его легкомысленным отношением к смерти, заявляется зама «Смерть», чтобы наказать юношу за такой проступок. Смерть заманивает его в свою смертоносную игру, откуда живым практически не выйти. Герою предстоит пережить свою смерть снова и снова в 13ти других жизнях. Если И Джэ сможет найти способ пережить неминуемую смерть, наступающую каждый раз, ему удастся остановить череду реинкарнаций и прожить свою жизнь. Его первая жизнь была ужасна и провалена, а как насчёт остальных? Сможет ли он осознать значение существования и найти верный путь для счастья и любви? Или ему придётся и дальше испытывать серию неудач, пока не попадёт в ад?",
        "Ким До Ги - бывший офицер спецназа, работающий водителем в службе такси. Он уникальный таксист, сочетающий в себе врождённую интуицию, здравомыслие, смелость, а также навыки боевых искусств, которые могут помочь ему выстоять против множества противников. Кан Ха На - прокурор, которая, как бульдозер, борется за справедливость. Вооружённая элитным образованием и честностью, у неё есть один недостаток - склонность проявлять агрессию при столкновении с несправедливостью. Во время расследования исчезновения бывшего заключенного у неё возникают подозрения относительно личности таинственного таксиста Ким До Ги. Чан Сон Чхоль - генеральный директор таинственной службы такси. Он возглавляет операцию по наказанию тех, кто совершает незаконные действия и пользуется преимуществами в слепой зоне государственной системы правосудия. Он теплее, чем кто-либо другой относится к жертвам, но абсолютно жесток и безжалостен по отношению к преступникам. Го Ын - хакер в службе такси, умеющая находить личную информацию людей. Она является экспертом в использовании всех цифровых устройств, от систем видеонаблюдения, смартфонов до компьютеров, и будет действовать как глаза и уши Ким До Ги.",
        "Семья девушки Сан Ми была вынуждена поселиться в южнокорейском уезде Муджу из-за финансовых проблем отца. Переезд никому не дался легко, а по потом ещё и выяснилось, что им продали не дом, а натуральные развалины. В дораме «Спаси меня» весь городок словно паутиной окутывает влияние религиозной секты под предводительством скользкого типа, называющего себя святым отцом. Он сразу положил глаз на Сан Ми и даже подсобил её отцу с жильём. Жизнь снова налаживается, когда Сан Ми вместе с братом-близнецом идёт в школу, где встречает хороших людей. Однако брату не везет: юноша сталкивается с травлей со стороны одноклассников. В конце концов они его настолько доводят, что тот совершает самоубийство."
]

russian_stopwords = stopwords.words('russian')
stemmer = SnowballStemmer('russian')

def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'\d+', '', text)
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in russian_stopwords and len(t) > 2]
    stemmed = [stemmer.stem(t) for t in tokens]
    return " ".join(stemmed)

preprocessed_texts = [preprocess_text(doc) for doc in texts]

print("Предобработанные синопсисы:")
for i, doc in enumerate(preprocessed_texts, 1):
    print(f"{i}: {doc}")

count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(preprocessed_texts)
bow_feature_names = count_vectorizer.get_feature_names_out()

bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=bow_feature_names,
    index=[f"Синопсис {i}" for i in range(1, 11)]
)

print("\nМатрица Bag of Words:\n")
print(bow_df)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_texts)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_feature_names,
    index=[f"Синопсис {i}" for i in range(1, 11)]
)
print("\nМатрица TF-IDF: \n")
print(tfidf_df)

def top_terms_per_doc(matrix, feature_names, top_n=10, is_tfidf=False):
    tops = {}
    for i in range(matrix.shape[0]):
        row = matrix[i].toarray().flatten()
        idx_sorted = np.argsort(row)[::-1]
        top_idx = idx_sorted[:top_n]
        if is_tfidf:
            tops[i] = [(feature_names[j], round(row[j], 4)) for j in top_idx if row[j] > 0]
        else:
            tops[i] = [(feature_names[j], int(row[j])) for j in top_idx if row[j] > 0]
    return tops

bow_top_per_doc = top_terms_per_doc(bow_matrix, bow_feature_names)
tfidf_top_per_doc = top_terms_per_doc(tfidf_matrix, tfidf_feature_names, is_tfidf=True)

print("\nТоп-10 BoW по отзывам:")
for i, terms in bow_top_per_doc.items():
    print(f"\nОтзыв {i+1}:")
    for w, v in terms:
        print("  -", w, v)

print("\nТоп-10 TF-IDF по отзывам:")
for i, terms in tfidf_top_per_doc.items():
    print(f"\nОтзыв {i+1}:")
    for w, v in terms:
        print("  -", w, v)

bow_cosine = cosine_similarity(bow_matrix)
tfidf_cosine = cosine_similarity(tfidf_matrix)

print("\nКосинусное сходство BoW:")
print(pd.DataFrame(bow_cosine.round(3)))

print("\nКосинусное сходство TF-IDF:")
print(pd.DataFrame(tfidf_cosine.round(3)))

bow_sum = np.sum(bow_matrix.toarray(), axis=0)
tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)

bow_top_corpus = sorted(zip(bow_feature_names, bow_sum), key=lambda x: x[1], reverse=True)[:10]
tfidf_top_corpus = sorted(zip(tfidf_feature_names, tfidf_sum), key=lambda x: x[1], reverse=True)[:10]

print("\nТОП-10 BoW по корпусу:")
for i, (w, f) in enumerate(bow_top_corpus, 1):
    print(f"{i:2d}. {w:<15} {f}")

print("\nТОП-10 TF-IDF по корпусу:")
for i, (w, s) in enumerate(tfidf_top_corpus, 1):
    print(f"{i:2d}. {w:<15} {s:.4f}")

bow_words = {w for w, _ in bow_top_corpus}
tfidf_words = {w for w, _ in tfidf_top_corpus}
common = bow_words & tfidf_words
bow_only = bow_words - tfidf_words
tfidf_only = tfidf_words - bow_words

print("\nОбщие слова (BoW и TF-IDF):", ", ".join(sorted(common)))
print("Только BoW:", ", ".join(sorted(bow_only)))
print("Только TF-IDF:", ", ".join(sorted(tfidf_only)))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Предобработанные синопсисы:
1: реальн жизн редк дел черн бел люд исключительн плох хорош дорам поведа истор двух совершен разн люд ëл плох парен потеря вер человечеств счита отличн полицейск успешн раскрыва одн дел друг однак встреч полн противоположн праведн человек скрыва псевдоним ке творя ужасн вещ полност меня ход событ ëл получ лиш аттестат средн образован однак помеша харизматичн обаятельн молод человек дослуж полиц ведущ сотрудник отдел борьб распространен запрещен веществ однак движет геро стремлен справедлив мужчин амбициоз предпочита закрыва глаз преступлен раскрыт котор способн принест продвижен карьерн лестниц одобрен начальств сам момент горизонт замаяч долгожда повышен перспективн мест город объяв сумасшедш преступник светл будущ стал ускольза ëл гот выйт троп войн стал разобра враг котор помеша план случ интерес обо пересекут стремлен одн то цел
2: сюжет дорам мыш раскрыва захватыва истор действ котор происход мир котор люд могут идентифицирова психопат помощ днктест пл

**Задание 2: Морфологическая разметка текста**

1. Возьмите отрывок текста (минимум 300 слов) на русском и английском языке.
2. Для русского языка используйте PyMorphy2 или PyMorphy3, для английского - NLTK или SpaCy для проведения морфологического анализа.
3. Выполните следующие операции:
* Определите части речи для каждого слова в тексте
* Для существительных определите падеж, род и число
* Для глаголов определите время, лицо и число
* Создайте частотный словарь частей речи в тексте
4. Разработайте функцию, которая будет автоматически изменять текст, заменяя все существительные на их форму множественного числа (где возможно).
5. Результаты морфологического анализа должны быть представлены в виде таблицы.
6. Оцените и прокомментируйте точность определения морфологических характеристик

In [13]:
!pip install inflect

import re
import nltk
import pymorphy3
import inflect
import spacy
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

russian_text = """В подтверждение отхожу от моего прежнего дома и бреду в центр. Гейл хотел высадиться вместе со мной, но не настаивал, когда я отказалась. Понимает: любая компания мне сегодня в тягость. Даже его. Есть дороги, которые нужно пройти в одиночку.
Лето выдалось сухим и знойным. Редкие дожди не смыли кучи золы. Она вспархивает от моих шагов и тут же оседает. Ветра нет. Я внимательно смотрю под ноги. Здесь раньше была дорога. Высадившись на Луговине, я споткнулась о камень. Только оказалось, что это не камень, а человеческий череп. Он откатился, зияющие глазницы уставились в небо. Я долго не могла отвести взгляд от зубов, гадая, кому они принадлежали, и представляя, как выглядели бы мои.
По привычке стараюсь придерживаться дороги, но она усеяна останками людей, пытавшихся спастись бегством. Некоторые сгорели полностью, другие, избежав огня, задохнулись от дыма. На полуразложившихся трупах кишат мухи. Вас всех убила я. И тебя. И тебя. И тебя…
Потому что так и есть. Моя стрела, разрушившая силовое поле, вызвала огненный шквал возмездия и повергла в хаос весь Панем.
В голове звучат слова президента Сноу, сказанные мне перед туром победителей: «Вы, Огненная Китнисс, бросили искру, способную разгореться в адское пламя, которое уничтожит Панем». Он не преувеличивал. Возможно, он искренне пытался заручиться моей поддержкой, хотя от меня уже ничего не зависело.
Пожар все еще продолжается, машинально отмечаю я. Угольные шахты вдали изрыгают клубы черного дыма. Впрочем, кому какое дело? Почти все жители дистрикта погибли. Оставшиеся восемь процентов укрылись в Тринадцатом дистрикте — бесприютные беженцы, навсегда лишившиеся родины.
Конечно, так думать не следует. Я должна быть благодарна за то, что нас приняли. Больных, израненных, голодных, нищих. Однако мне не дает покоя мысль, что, если бы не мятежники Тринадцатого дистрикта, Двенадцатый был бы цел. Это не снимает вины с меня — ее хватит на всех. Но что бы я смогла без них?
Жители Двенадцатого дистрикта вообще ни в чем не виноваты, им просто не повезло, что у них есть я. И все же некоторые выжившие рады, что вырвались оттуда. Прочь от голода и гнета, гибельных шахт, от плети начальника миротворцев Ромулуса Треда. Крыша над головой — уже счастье, ведь до недавнего времени мы и не подозревали о существовании Тринадцатого дистрикта."""

english_text = """To reinforce this, I begin to move away from my old house and in toward the town. Gale asked to be dropped off in 12 with me, but he didn't force the issue when I refused his company. He understands I don't want anyone with me today. Not even him. Some walks you have to take alone. The summer's been scorching hot and dry as a bone. There's been next to no rain to disturb the piles of ash left by the attack. They shift here and there, in reaction to my footsteps. No breeze to scatter them. I keep my eyes on what I remember as the road, because when I first landed in the Meadow, I wasn't careful and I walked right into a rock. Only it wasn't a rock--it was someone's skull. It rolled over and over and landed faceup, and for a long time I couldn't stop looking at the teeth, wondering whose they were, thinking of how mine would probably look the same way under similar circumstances. I stick to the road out of habit, but it's a bad choice, because it's full of the remains of those who tried to flee. Some were incinerated entirely. But others, probably overcome with smoke, escaped the worst of the flames and now lie reeking in various states of decomposition, carrion for scavengers, blanketed by flies. I killed you, I think as I pass a pile. And you. And you. Because I did. It was my arrow, aimed at the chink in the force field surrounding the arena, that brought on this firestorm of retribution. That sent the whole country of Panem into chaos. In my head I hear President Snow's words, spoken the morning I was to begin the Victory Tour. "Katniss Everdeen, the girl who was on fire, you have provided a spark that, left unattended, may grow to an inferno that destroys Panem." It turns out he wasn't exaggerating or simply trying to scare me. """

eng_tokens = word_tokenize(english_text)
eng_tagged = pos_tag(eng_tokens)

eng_tagged_joined = ["_".join(pair) for pair in eng_tagged]

eng_tokens_np = [t for t in eng_tokens if re.search(r"[A-Za-z0-9]", t)]
eng_pos_counts = Counter(tag for _, tag in pos_tag(eng_tokens_np))

eng_df = pd.DataFrame(eng_tagged, columns=["token", "pos_tag"])
eng_freq_df = pd.DataFrame(
    eng_pos_counts.most_common(),
    columns=["pos_tag", "count"]
)

morph = pymorphy3.MorphAnalyzer()
rus_tokens = word_tokenize(russian_text)

rus_list_of_tagged = []
rus_pos_counts = Counter()
rus_rows = []

for word in rus_tokens:
    if all(not ch.isalnum() and ch not in "ёЁ" for ch in word):
        continue

    p = morph.parse(word)[0]
    pos = p.tag.POS or "UNKN"
    rus_pos_counts[pos] += 1

    if pos == "NOUN":
        rus_list_of_tagged.append(
            f"{word}_{pos}_{p.tag.case}_{p.tag.gender}_{p.tag.number}"
        )
    elif pos == "VERB":
        rus_list_of_tagged.append(
            f"{word}_{pos}_{p.tag.person}_{p.tag.tense}_{p.tag.number}"
        )
    else:
        rus_list_of_tagged.append(f"{word}_{pos}")

    rus_rows.append({
        "token": word,
        "pos_tag": pos,
        "case": p.tag.case,
        "gender": p.tag.gender,
        "number": p.tag.number,
        "person": p.tag.person,
        "tense": p.tag.tense,
    })

rus_df = pd.DataFrame(rus_rows)
rus_freq_df = pd.DataFrame(
    rus_pos_counts.most_common(),
    columns=["pos_tag", "count"]
)

p_inflect = inflect.engine()
nlp_en = spacy.load("en_core_web_sm")

def pluralize_nouns_pymorphy():
    new_tokens = []

    for w in rus_tokens:
        if any(ch.isalpha() for ch in w):
            p = morph.parse(w)[0]
            if p.tag.POS == "NOUN" and p.tag.number == "sing":
                gramm = {"plur"}
                if p.tag.case:
                    gramm.add(p.tag.case)
                if p.tag.animacy:
                    gramm.add(p.tag.animacy)

                inf = p.inflect(gramm)
                if inf:
                    w = inf.word
        new_tokens.append(w)

    return new_tokens

russian_pluralized = detok.detokenize(pluralize_nouns_pymorphy())

def pluralize_nouns_spacy(text: str) -> str:
    doc = nlp_en(text)
    out = []

    for tok in doc:
        if tok.tag_ in ("NN", "NNP"):
            pl = p_inflect.plural_noun(tok.text) or p_inflect.plural(tok.text)

            if tok.text.isupper():
                pl = pl.upper()
            elif tok.text.istitle():
                pl = pl.title()

            out.append(pl + tok.whitespace_)
        else:
            out.append(tok.text_with_ws)

    return "".join(out)

english_pluralized = pluralize_nouns_spacy(english_text)

detok = TreebankWordDetokenizer()

print("\nRUS tagged tokens:")
print(rus_list_of_tagged)

print("ENG tagged tokens:")
print(eng_tagged_joined)

print("\nRUS POS frequency:")
print(rus_freq_df)


print("\nENG POS frequency:")
print(eng_freq_df)

print("\nRUS tagged table:")
print(rus_df)

print("\nENG tagged table:")
print(eng_df)

print("\nRussian pluralized text:")
print(russian_pluralized)

print("\nEnglish pluralized text:")
print(english_pluralized)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



RUS tagged tokens:
['В_PREP', 'подтверждение_NOUN_nomn_neut_sing', 'отхожу_VERB_1per_futr_sing', 'от_PREP', 'моего_ADJF', 'прежнего_ADJF', 'дома_NOUN_gent_masc_sing', 'и_CONJ', 'бреду_NOUN_accs_femn_sing', 'в_PREP', 'центр_NOUN_nomn_masc_sing', 'Гейл_NOUN_nomn_masc_sing', 'хотел_VERB_None_past_sing', 'высадиться_INFN', 'вместе_ADVB', 'со_PREP', 'мной_NPRO', 'но_CONJ', 'не_PRCL', 'настаивал_VERB_None_past_sing', 'когда_CONJ', 'я_NPRO', 'отказалась_VERB_None_past_sing', 'Понимает_VERB_3per_pres_sing', 'любая_ADJF', 'компания_NOUN_nomn_femn_sing', 'мне_NPRO', 'сегодня_ADVB', 'в_PREP', 'тягость_NOUN_accs_femn_sing', 'Даже_PRCL', 'его_NPRO', 'Есть_INFN', 'дороги_NOUN_gent_femn_sing', 'которые_ADJF', 'нужно_PRED', 'пройти_INFN', 'в_PREP', 'одиночку_NOUN_accs_femn_sing', 'Лето_NOUN_accs_neut_sing', 'выдалось_VERB_None_past_sing', 'сухим_ADJF', 'и_CONJ', 'знойным_ADJF', 'Редкие_ADJF', 'дожди_NOUN_nomn_masc_plur', 'не_PRCL', 'смыли_VERB_None_past_plur', 'кучи_NOUN_gent_femn_sing', 'золы_NOUN_g

**Задание 3: Синтаксический анализ предложений**

1. Возьмите 2 простых и 3 сложных предложений на русском и английском языке (всего 10 предложений).
2. Используйте SpaCy для построения синтаксических деревьев этих предложений.
3. Для каждого предложения:
* Визуализируйте синтаксическое дерево
* Выделите все подлежащие и сказуемые
* Найдите все пары слов, связанные отношением определения (прилагательное-существительное)
4. Разработайте функцию для извлечения всех объектных и субъектных отношений из предложения в формате (субъект, предикат, объект).
5. Объясните, какие трудности возникают при синтаксическом анализе сложных предложений

In [20]:
!python -m spacy download ru_core_news_md
!python -m spacy download en_core_web_md

nlp_ru = spacy.load("ru_core_news_md")
nlp_en = spacy.load("en_core_web_md")
from spacy import displacy

russian_sentences = [
    "Я гуляю по улице.",
    "Я люблю хорошую погоду.",
    "Когда я вышел на улицу, начался сильный дождь, хотя утром светило солнце.",
    "Девушка, которая учится в университете, написала интересную статью.",
    "Если бы он знал правду, то не принял бы такое решение, но теперь уже ничего не изменить."
]

english_sentences = [
    "The cat sleeps on the mat.",
    "She reads interesting books.",
    "Although it was raining heavily, we decided to go for a walk because we had promised our friends.",
    "The book that I read yesterday, which was written by a famous author, contains many fascinating stories about ancient civilizations.",
    "If I had known about the meeting earlier, I would have prepared better, but unfortunately nobody informed me in time."
]

print('Визуализация синтакисческого дерева для русского языка\n')
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)

    print(f"Предложение {russian_sentences.index(sentence) + 1}")
    displacy.render(doc_ru, style="dep", jupyter=True, options={"distance": 100, "compact": True})

print('Визуализация синтакисческого дерева для английского языка\n')
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    print(f"Sentence {english_sentences.index(sentence) + 1}")
    displacy.render(doc_en, style="dep", jupyter=True, options={"distance": 100, "compact": True})

def find_subj_pred_rus(doc):
    root = None
    pred_tokens = []

    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break

    if not root:
        return [], []

    changed = True
    while changed:
        changed = False
        for t in doc:
            if t.dep_ == "conj" and t.head in pred_tokens and t.pos_ in {"VERB", "AUX"}:
                if t not in pred_tokens:
                    pred_tokens.append(t)
                    changed = True

    def pred_text(p):
        parts = {p}
        for ch in p.children:
            if ch.dep_ in {"aux", "auxpass", "cop", "neg"}:
                parts.add(ch)
            if ch.dep_ == "xcomp" and ch.pos_ in {"VERB", "AUX"}:
                parts.add(ch)
        return " ".join(tok.text for tok in sorted(parts, key=lambda x: x.i))

    def subjects(p):
        subs = [ch for ch in p.children if ch.dep_ in {"nsubj", "nsubjpass", "nsubj:pass"}]
        if subs:
            return subs
        if p.dep_ == "conj":
            return subjects(p.head)
        return []

    all_subj = []
    all_pred = []
    for p in sorted(pred_tokens, key=lambda x: x.i):
        all_pred.append(pred_text(p))
        all_subj.extend([s.text for s in subjects(p)])

    return all_subj, all_pred
for text in russian_sentences:
  doc = nlp_ru(text)
  subject, predicate = find_subj_pred_rus(doc)
  print(f"Предложение: {text}")
  print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
  print()


def find_subj_pred_eng(doc):
    subj = []
    pred_tokens = []
    root = None
    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break
    if not root:
        return [], []

    for token in doc:
        if token.dep_ in ["aux", "auxpass"] and token.head in [root] + [t for t in pred_tokens]:
            pred_tokens.append(token)
    pred = [doc[i].text for i in sorted([t.i for t in pred_tokens])]
    for token in doc:
        if token.dep_ in ["nsubj", "nsubjpass"]:
            current = token
            while current.head != current:
                if current.head in pred_tokens or current.head == root:
                    subj.append(token.text)
                    break
                current = current.head
    if not subj:
        for token in doc:
            if token.dep_ in ["nsubj", "nsubj:pass"]:
                subj.append(token.text)
    return subj, pred
for text in english_sentences:
    doc = nlp_en(text)
    subject, predicate = find_subj_pred_eng(doc)
    print(f"Предложение: {text}")
    print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
    print()

print("Отношение прилагательное-существительное в русских предложениях:")
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)
    for token in doc_ru:
        if token.dep_ == "amod":
            print(f"Объект '{token.text}' зависит от существительного '{token.head.text}'")

print("\nОтношение прилагательное-существительное в английских предложениях:")
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    for token in doc_en:
        if token.dep_ == "amod":
            print(f"Объект '{token.text}' зависит от существительного '{token.head.text}'")



     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
      --------------------------------------- 0.5/41.9 MB 1.9 MB/s eta 0:00:23
     - -------------------------------------- 1.6/41.9 MB 3.6 MB/s eta 0:00:12
     -- ------------------------------------- 2.6/41.9 MB 4.6 MB/s eta 0:00:09
     --- ------------------------------------ 3.1/41.9 MB 3.8 MB/s eta 0:00:11
     ----- ---------------------------------- 5.2/41.9 MB 5.2 MB/s eta 0:00:08
     ----- ---------------------------------- 6.0/41.9 MB 5.3 MB/s eta 0:00:07
     -------- ------------------------------- 8.7/41.9 MB 6.1 MB/s eta 0:00:06
     ---------- ----------------------------- 11.3/41.9 MB 6.8 MB/s eta 0:00:05
     ------------ --------------------------- 13.1/41.9 MB 7.0 MB/s eta 0:00:05
     -------------- ------------------------- 15.2/41.9 MB 7.2 MB/s eta

ImportError: cannot import name 'display' from 'IPython.core.display' (c:\Users\Home\LancsBoxX\miniconda\Lib\site-packages\IPython\core\display.py)

**Задание 4: Распознавание именованных сущностей (Named Entity Recognition)**

1. Подготовьте корпус из 10 новостных текстов, содержащий различные типы именованных сущностей (имена людей, организации, географические названия, даты и т.д.) на английском или русском языке.
2. Используйте SpaCy для автоматического распознавания именованных сущностей.
3. Реализуйте свой простой метод для распознавания имен людей и географических названий с помощью регулярных выражений и словарей.
4. Сравните результаты работы SpaCy и вашего метода:
* Рассчитайте точность (precision), полноту (recall) и F1-меру для вашего метода относительно результатов SpaCy
* Проанализируйте ошибки обоих подходов, какие типы ошибок характерны для каждого подхода
5. Представьте сравнение результатов в виде таблицы

In [35]:
documents = [
    "Friends and Family of Valentino Bid Farewell to Fashion Giant. Famous faces from the worlds of fashion and film descended on Rome on Friday to bid farewell to legendary Italian designer Valentino Garavani, who died on Monday at 93."
    "For his latest tour, Harry Styles is coming to Madison Square Garden. A lot. Styles, who is set to release his fourth solo album, “Kiss All the Time. Disco, Occasionally.,” on March 6, will perform a 30-show residency at the Garden, appearing every Wednesday, Friday and Saturday from Aug. 26 to Oct. 31, Live Nation, the tour’s promoter, announced on Thursday. The residency will be the only United States dates this year on Styles’s 50-stop world tour, called Together, Together, which is set to begin in Amsterdam in May. Information about ticket sales is available on Styles’s website. The full tour involves shows in seven cities, also including São Paulo, Mexico City, Melbourne, Sydney and London, where he will play six nights at Wembley Stadium. Opening acts, which vary by city, include Jamie xx, Robyn and Shania Twain."
    "ASAP Rocky Grew Up, Settled Down (with Rihanna) and Returned to Rap. The artist reflects on how the chaotic eight years since his last release — including three kids and two trials — led to his latest album, Don’t Be Dumb. In the time since ASAP Rocky released his last album, “Testing,” in 2018, he’s acted in two Hollywood movies, become romantic partners and a father of three with Rihanna, faced two criminal prosecutions and seen dozens of his unfinished songs let loose onto the internet by leakers. It has been a sometimes agonizing, sometimes thrilling stretch of time defined by upheaval, and it sometimes seemed like Rocky might never put out an album again. “I never accepted that,” he said on Popcast, The New York Times pop culture show, in an interview last week. “That’s my core. I consider myself a Renaissance man, so it starts with music.” His fourth album, “Don’t Be Dumb,” released Friday, is a return to some of Rocky’s familiar musical influences, with gothic takes on Southern rap styles (“Playa,” “Helicopter”) and experimental production that dabbles in cloudy abstraction and psychedelic rock (“Punk Rocky”). Typical of the rapper’s preferred eclecticism, the album features appearances by rappers like Doechii and Tyler, the Creator, alongside vocals from the folk singer Jessica Pratt and Damon Albarn."
    "On the cover of Danny Brown’s new album, “Stardust,” the rapper’s face appears in shadow, but his well-defined abs are clearly visible. Is the 44-year-old excited to show off the results of his hard work at the gym? “Totally, totally,” he said. In 2023, Brown entered rehab, and he’s eager to discuss sobriety’s positive impact on his physical and mental health, his relationships and his artmaking process. Brown is a Detroit native with a classicist’s appreciation for language, but an ear for beats that are anything but traditional. Years ago he heard the glitchy sounds of 100 gecs and thought that he could rap over that, or something like it, and began to explore the experimental pop scene. His collaborators on his sixth album — including the genre-resistant New Jersey radical Jane Remover, the sibling duo Frost Children and the alt-rock-adjacent artist Underscores — have little history of working with a rapper like Brown. They all joined a group chat where he would share Dropbox links to in-progress material. His only prompt: “What would you want a Danny Brown song to sound like in the mix of these?”"
    "If the first two episodes of “The End of an Era” — the documentary series about Taylor Swift’s record-breaking Eras Tour — were defined by how Swift weathered the brutal moments that befell her road show, the final four are marked by the pop star’s joy. The last batch of episodes, now all available to stream on Disney+, chart how Swift rebounded from not one but two heartbreaks and embarked on a relationship with her fiancé, Travis Kelce. Along the way, she recorded yet another album in secret and completed the record-breaking concert extravaganza. Though the Kelce details are perhaps the most tantalizing, Swift also offers more insight into the complexities and challenges of mounting a tour of that scale."
    "Brazil’s Latest Icon Is New to Film. Now, She’s a Star of an Oscar Contender. Clad in florals and puffing on a cigarette, Tânia Maria has captivated audiences with a striking, if brief, performance in “The Secret Agent,” Brazil’s latest Oscar nominee."
    "Despite Trump’s Words, China and Russia Are Not Threatening Greenland U.S. and European officials say they are unaware of any intelligence that shows China and Russia are endangering the island, which is protected by the NATO security umbrella."
    "McLaren awarded more than $12m in legal case against IndyCar champion Alex Palou. As previously reported by The Athletic, the dispute dates back to October 2022, when Palou agreed to sign for McLaren’s IndyCar team, Arrow McLaren. Under that deal, it was agreed Palou would take up a position as McLaren’s reserve Formula 1 driver in 2023, with the possibility of promotion, before racing for their IndyCar team across the 2024, 2025 and 2026 seasons. He would also continue to drive for his existing Chip Ganassi Racing (CGR) team in the 2023 season. However, in August 2023, Palou opted against making the move, instead choosing to stay with CGR. McLaren later sued Palou, claiming in excess of $20m in losses, according to Friday’s judgement. In total, McLaren was awarded $10.2m, with a further amount of between $2m and $2.5m dependent on the outcome of expert evidence."
    "The K-pop band NewJeans plunged deeper into turmoil this week after its management label said it had dropped one member of the five-person girl group, one of the genre’s most influential acts in recent years. The announcement on Monday by the group’s label, Ador, that it was terminating the contract of Danielle Marsh, who performs as Danielle, is the latest flashpoint in a more than yearlong legal battle between the label and the band. The label did not provide a reason for dropping Danielle. It said that three NewJeans members — Haerin, Hyein and Hanni — would continue working while it continued “discussions” with Minji, a fourth member. Neither Danielle nor Minji responded immediately to requests for comment on Tuesday. It is unclear when or whether NewJeans will record music or perform as a group again. Even though K-pop labels often let go of artists, Danielle’s dismissal has prompted intense scrutiny and news coverage in South Korea. That’s because it illustrates the complexities of management structures and contracts in a largely top-down industry that is highly restrictive and led by a handful of powerful entertainment conglomerates."
    "Rebecca Hall had never worked with Ryan Murphy when he pitched her what she called “the most wild and fun and subversive wacky premise” for a new show titled The Beauty. Hall admitted to existing in a realm of “permanent tension” between her desire to age gracefully and naturally, and appreciating that she works in an industry where how she looks matters. In a video call from her home in upstate New York, Hall spoke about her mother’s piano, the brown and green cocktails her husband Morgan Spector makes, and things that go bump in the night. These are edited excerpts from the conversation."
]

In [36]:
import spacy
import re
import pandas as pd

nlp = spacy.load("en_core_web_md")

In [39]:
def extract_spacy_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def simple_ner(text):
    entities = []
    known_names = [
        'ASAP Rocky', 'Valentino Garavani', 'Harry Styles',
        'Shania Twain', 'Jane Remover', 'Danny Brown',
        'Frost Children', 'Underscores', 'Taylor Swift',
        'Travis Kelce', 'Tânia Maria', 'Trump',
        'NATO', 'Arrow McLaren', 'Alex Palou',
        'New Jeans', 'Danielle Marsh', 'Ador',
        'Haerin', 'Hyein', 'Hanni',
        'Minji', 'Rebecca Hall', 'Ryan Murphy',
        'Morgan Spector', 'Travis Kelce', 'Rihanna', 'Doechii',
        'Tyler, the Creator', 'Jessica Pratt', 'Damon Albarn'
    ]
    known_locations = [
        'New York', 'China', 'Russia', 'South Korea',
        'Detroit', 'New Jersey', 'São Paulo', 'Mexico City',
        'Melbourne', 'Sydney', 'London', 'Brazil', 'Greenland',
        'United States', 'Amsterdam'
    ]

    name_patterns = [
        r'\b([A-Z][a-z]+\s+[A-Z][a-z]+)\b',
        r'\b([A-Z][a-z]+\s+[A-Z]\.\s+[A-Z][a-z]+)\b',
        r'\b([A-Z][a-z]+\s+[A-Z][a-z]+\s+[A-Z][a-z]+)\b'
    ]

    location_patterns = [
        r'\b([A-Z][a-z]+\s+[A-Z][a-z]+)\b',
        r'\b(?:in|at|from|to)\s+([A-Z][a-z]+)\b',
        r'\b([A-Z][a-z]+\s+(?:City|State|County|Island|River|Lake|Mount))\b'
    ]

    for name in known_names:
        if name in text:
            entities.append((name, 'PERSON'))

    for location in known_locations:
        if location in text:
            entities.append((location, 'LOC'))

    for pattern in name_patterns:
        for match in re.finditer(pattern, text):
            name = match.group()
            if name not in [e[0] for e in entities]:
                entities.append((name, 'PERSON'))

    for pattern in location_patterns:
        for match in re.finditer(pattern, text):
            location = match.group()
            if location not in [e[0] for e in entities]:
                entities.append((location, 'LOC'))

    unique_entities = []
    seen = set()
    for ent in entities:
        if ent[0] not in seen:
            unique_entities.append(ent)
            seen.add(ent[0])

    return unique_entities

def calculate_metrics(predicted, reference):
    predicted_set = set([entity[0].lower() for entity in predicted])
    reference_set = set([entity[0].lower() for entity in reference])

    true_positives = len(predicted_set.intersection(reference_set))
    false_positives = len(predicted_set - reference_set)
    false_negatives = len(reference_set - predicted_set)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1, true_positives, false_positives, false_negatives

spacy_results = []
simple_results = []
metrics = []

print("\nСущности, которые распознал каждый метод:\n")

for i, document in enumerate(documents):
    print(f"\n{'='*80}")
    print(f"Документ {i+1}:")
    print(f"{document[:150]}..." if len(document) > 150 else document)

    spacy_entities = extract_spacy_entities(document)
    spacy_results.append(spacy_entities)

    simple_entities = simple_ner(document)
    simple_results.append(simple_entities)

    print(f"\nSpaCy ({len(spacy_entities)} сущностей):")
    for entity, label in spacy_entities:
        print(f"  {entity:40} [{label}]")

    print(f"\nПростой метод ({len(simple_entities)} сущностей):")
    for entity, label in simple_entities:
        print(f"  {entity:40} [{label}]")

    precision, recall, f1, tp, fp, fn = calculate_metrics(simple_entities, spacy_entities)
    metrics.append({
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': tp,
        'fp': fp,
        'fn': fn
    })

    print(f"\nМетрики для документа {i+1}:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1: {f1:.3f}")
    print(f"  Совпадения (TP): {tp}")
    print(f"  Ложные срабатывания (FP): {fp}")
    print(f"  Пропущенные (FN): {fn}")

print("\n" + "="*80)
print("Общая таблица полученных результатов:\n")

table_data = []
for i in range(len(documents)):
    table_data.append({
        'Документ': i+1,
        'Precision': f"{metrics[i]['precision']:.3f}",
        'Recall': f"{metrics[i]['recall']:.3f}",
        'F1': f"{metrics[i]['f1']:.3f}",
        'TP': metrics[i]['tp'],
        'FP': metrics[i]['fp'],
        'FN': metrics[i]['fn'],
        'Сущностей (spaCy)': len(spacy_results[i]),
        'Сущностей (простой)': len(simple_results[i])
    })

avg_precision = sum(m['precision'] for m in metrics) / len(metrics)
avg_recall = sum(m['recall'] for m in metrics) / len(metrics)
avg_f1 = sum(m['f1'] for m in metrics) / len(metrics)
total_tp = sum(m['tp'] for m in metrics)
total_fp = sum(m['fp'] for m in metrics)
total_fn = sum(m['fn'] for m in metrics)
total_spacy = sum(len(r) for r in spacy_results)
total_simple = sum(len(r) for r in simple_results)

table_data.append({
    'Документ': 'Среднее/Итого',
    'Precision': f"{avg_precision:.3f}",
    'Recall': f"{avg_recall:.3f}",
    'F1': f"{avg_f1:.3f}",
    'TP': total_tp,
    'FP': total_fp,
    'FN': total_fn,
    'Сущностей (spaCy)': total_spacy,
    'Сущностей (простой)': total_simple
})

df = pd.DataFrame(table_data)
print(df.to_string(index=False))


Сущности, которые распознал каждый метод:


Документ 1:
Friends and Family of Valentino Bid Farewell to Fashion Giant. Famous faces from the worlds of fashion and film descended on Rome on Friday to bid far...

SpaCy (166 сущностей):
  Rome                                     [GPE]
  Friday                                   [DATE]
  Italian                                  [NORP]
  Valentino Garavani                       [PERSON]
  Monday                                   [DATE]
  93.For                                   [DATE]
  Harry Styles                             [PERSON]
  Madison Square Garden                    [FAC]
  fourth                                   [ORDINAL]
  Kiss All the Time                        [WORK_OF_ART]
  Disco                                    [PERSON]
  March 6                                  [DATE]
  30                                       [CARDINAL]
  Wednesday, Friday and                    [DATE]
  Saturday                                 [DAT